# Møller--Plesset

## Møller--Plesset partitioning

In molecular electronic structure theory, $\hat{H}$ is the Born-Oppenheimer, many-body, molecular electronic Hamiltonian. However, in general, the partitioning of the Hamiltonian, i.e., the definition of $\hat{H}_0$, can be achieved in a number of ways.  
%For example, taking the diagonal of the Hamiltonian in a basis of SDs as $\hat{H}_0$, we see that the [first-order amplitudes](eq:PT_amp1) become identical to the first Davidson iteration in a configuration interaction calculation, see [this equation](eq:cidav).

The most common partitioning in quantum chemistry is probably the Møller--Plesset (MP) partitioning {cite}`Szabo2012,Helgaker2014`, since it follows quite naturally when first approximating the desired ground state in terms of a *single reference determinant*. The Hamiltonian is in fact rewritten as:

$$
\hat{H} = \hat{F} + \hat{\Phi}
$$

where $\hat{F}$ is the Fock operator and $\hat{\Phi}$ is the so-called *fluctuation potential*. $\hat{F}$ is a one-body operator whose spectrum consists of all determinants that can be built from excitation of the reference $| 0 \rangle$:

$$
\hat{F} | 0 \rangle  = E_{0}^{(0)}| 0 \rangle, \quad
\hat{F} \left|_{ij\ldots} ^{ab\ldots} \right\rangle =
\left(E_{0}^{(0)} + \varepsilon^{ab\cdots}_{ij\cdots}  \right)
\left|_{ij\ldots} ^{ab\ldots} \right\rangle,
$$

where the *zeroth-order energy* and the *orbital-energy denominators* are:

$$
E_{0}^{(0)} = \sum_{i} \varepsilon_{i},\quad
\varepsilon^{ab\cdots}_{ij\cdots} = \varepsilon_{a} + \varepsilon_{b} + \cdots - \varepsilon_{i} - \varepsilon_{j} - \cdots
$$

Here and in the following, the indices $i,j,\ldots$ refer to occupied orbitals, whereas $a,b,\ldots$ refer to unoccupied (virtual) ones.
The fluctuation potential is a two-body operator. With this partinioning, the zeroth-order energy is the sum of orbital energies. The first-order correction is:

$$
E_0^{(1)} = \left\langle 0 \left| \hat{\Phi} \right| 0 \right\rangle = -\frac{1}{2} \sum_{ij} \langle ij \| ij \rangle
$$

that is, the energy of the reference single determinant is *correct* through first order in the perturbative series: $E_{\mathrm{ref}} = E_{\mathrm{HF}} = E_{0}^{(0)} + E_0^{(1)}$.

(label:mp1-wf)=
The first-order correction to the wave function is obtained from the general RSPT expression. In a basis of molecular spin-orbitals, it is given by
$$
| \Psi_0^{(1)} \rangle = -\frac{1}{4}\sum_{ijab}  
\frac{\langle ab \| ij \rangle}{\varepsilon_{ij}^{ab}} |_{ij}^{ab}\rangle \, ,
$$

where the *orbital-energy denominator* is $\varepsilon_{ij}^{ab} = \varepsilon_{a} + \varepsilon_{b} - \varepsilon_{i} - \varepsilon_{j}$,
and the expansion coefficients are identified as the so-called $t$ _amplitudes_ with
$$
t_{ijab} = \frac{\langle ab \| ij \rangle}{\varepsilon_{ij}^{ab}} \, .
$$

The second-order energy correction follows:
$$
E_{0}^{(2)}  = - \frac{1}{4} 
\sum_{ijab} \frac{\langle ij \| ab \rangle \langle ab \| ij \rangle}{\varepsilon_{ij}^{ab}}
= - \frac{1}{4} \sum_{ijab} t_{ijab} \langle ij || ab \rangle
$$
and the total MP2 energy is given by $E_{\mathrm{MP2}} = E_{\mathrm{HF}} + E_0^{(2)}$.
For a closed-shell, restricted reference using real MOs, $E_0^{(2)}$ can be written as

$$
E_0^{(2)} = -
\sum_{ij}^{N_{\mathrm{O}}} \sum_{ab}^{N_{\mathrm{V}}} 
\frac{\langle ij | ab \rangle}{\varepsilon_{ij}^{ab}}
[ 2 \langle ij | ab \rangle - \langle ij | ba \rangle ],
$$
where $N_{\mathrm{O}}$ is the numer of occupied _spatial_ orbitals (corresponding to half the number of electrons),
and $N_{\mathrm{V}}$ the number of virtual spatial orbitals,
which we can further rearrange into to two terms, *opposite-spin* (OS) and *same-spin* (SS):

$$
E_0^{(2)} = -
\sum_{ij}^{N_{\mathrm{O}}} \sum_{ab}^{N_{\mathrm{V}}} 
\frac{\langle ij | ab \rangle\langle ij | ab \rangle}{\varepsilon_{ij}^{ab}} -
\sum_{ij}^{N_{\mathrm{O}}} \sum_{ab}^{N_{\mathrm{V}}} 
\frac{\langle ij | ab \rangle[ \langle ij | ab \rangle - \langle ij | ba \rangle ]}{\varepsilon_{ij}^{ab}} = 
E_{\mathrm{OS}}^{(2)} + E_{\mathrm{SS}}^{(2)}.
$$

## Implementation

To compute $E_0^{(2)}$ we need to:

1. Obtain the reference closed-shell determinant from a Hartree-Fock calculation.
2. Transform the AO basis ERI tensor to MO basis.
3. Assemble the energy denominators.
4. Combine the results of steps 2 and 3 to form the perturbative correction.

![Obtaining the MP2 energy correction](../img/mp2.svg)

### Obtaining the HF reference

We start with the declaration of the usual water molecule and its basis set. We also perform the SCF calculation with the `ScfRestrictedDriver`.

In [1]:
import veloxchem as vlx

h2o_xyz = """3
water                                                                                                                          
O    0.000000000000        0.000000000000        0.000000000000                         
H    0.000000000000        0.740848095288        0.582094932012                         
H    0.000000000000       -0.740848095288        0.582094932012
"""

mol = vlx.Molecule.from_xyz_string(h2o_xyz)

basis = vlx.MolecularBasis.read(mol, "cc-pvdz")

scfdrv = vlx.ScfRestrictedDriver()
scfdrv.compute(mol, basis)

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 4.
                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error

We can now access orbital energies and MO coefficients from the driver:

In [2]:
epsilon = scfdrv.scf_tensors["E"]
C = scfdrv.scf_tensors["C"]

### Transforming the integrals

We compute the MP2 energy correction with the ERI expressed in MO basis: we need to transform the ERI tensor from AO basis.
The transformation reads:

$$
\langle pq | rs \rangle = \sum_{\mu\nu\kappa\lambda} C_{\mu p}C_{\nu r} (\mu\nu|\kappa\lambda) C_{\kappa q} C_{\lambda s},
$$

with the MO integrals in [**physicists' notation**](http://vergil.chemistry.gatech.edu/notes/permsymm/permsymm.html). The transformation requires $O(N^{8})$ operation count. 
However, we can perform it more efficiently as a stepwise contraction:

$$
\langle pq | rs \rangle = \sum_{\mu} C_{\mu p}  \left(\sum_{\nu} C_{\nu r}  \left (\sum_{\kappa} \left(\sum_{\lambda} (\mu\nu|\kappa\lambda) C_{\lambda s} \right) C_{\kappa q} \right)\right).
$$

We should also note that we do **not** need the full ERI tensor in MO basis, but rather the *OOVV* class of integrals, which involve two occupied and two virtual MO indices:

$$
\langle ij | ab \rangle = 
\sum_{\mu} C_{\mu i}  
\left(\sum_{\nu} C_{\nu j}  
\left(\sum_{\kappa} 
\left(\sum_{\lambda} (\mu\kappa|\nu\lambda) C_{\lambda b} \right)
C_{\kappa a}\right)\right).
$$

In [3]:
eridrv = vlx.ElectronRepulsionIntegralsDriver()
mknl = eridrv.compute_in_mem(mol, basis)

In [8]:
import numpy as np

N_O = mol.number_of_electrons() // 2
N_V = scfdrv.mol_orbs.number_mos() - N_O

mknb = np.einsum("mknl,lB->mknB", mknl, C[:, N_O:])
print(f"mknb.shape={mknb.shape}")
mnab = np.einsum("mknB,kA->mnAB", mknb, C[:, N_O:])
print(f"mnab.shape={mnab.shape}")
mjab = np.einsum("mnAB,nJ->mJAB", mnab, C[:, :N_O])
print(f"mjab.shape={mjab.shape}")
ijab = np.einsum("mJAB,mI->IJAB", mjab, C[:, :N_O])
print(f"ijab.shape={ijab.shape}")

mknb.shape=(24, 24, 24, 19)
mnab.shape=(24, 24, 19, 19)
mjab.shape=(24, 5, 19, 19)
ijab.shape=(5, 5, 19, 19)


Let's compare our *OOVV* ERI tensor with the one computed by using VeloxChem's own `MOIntegralsDriver`:

In [ ]:
moeridrv = vlx.MOIntegralsDriver()
moeri = moeridrv.compute_in_mem(mol, basis, mol_orbs=scfdrv.mol_orbs, mints_type="OOVV")

np.testing.assert_allclose(ijab, moeri, atol=1.e-10)

### The MP2 energy correction

We now have all the ingredients to compute the *opposite-spin* and *same-spin* components of the MP2 energy correction:

$$
E_0^{(2)} = -
\sum_{ij}^{N_{\mathrm{O}}} \sum_{ab}^{N_{\mathrm{V}}} 
\frac{\langle ij | ab \rangle\langle ij | ab \rangle}{\varepsilon_{ij}^{ab}} - 
\sum_{ij}^{N_{\mathrm{O}}} \sum_{ab}^{N_{\mathrm{V}}} 
\frac{\langle ij | ab \rangle[ \langle ij | ab \rangle - \langle ij | ba \rangle ]}{\varepsilon_{ij}^{ab}} = 
E_{\mathrm{OS}}^{(2)} + E_{\mathrm{SS}}^{(2)}.
$$

In [ ]:
e_mp2_ss = 0.0
e_mp2_os = 0.0

# extract the occupied subset of the orbital energies
e_ij = epsilon[:N_O]
# extract the virtual subset of the orbital energies
e_ab = epsilon[N_O:]

for i in range(N_O):
    for j in range(N_O):
        for a in range(N_V):
            for b in range(N_V):
                # enegy denominators
                e_ijab = e_ab[a] + e_ab[b] - e_ij[i] - e_ij[j] 
                
                # update opposite-spin component of the energy
                e_mp2_os -= (ijab[i, j, a, b] * ijab[i, j, a, b]) / e_ijab
                
                # update same-spin component of the energy
                e_mp2_ss -= ijab[i, j, a, b] * (ijab[i, j, a, b]  - ijab[i, j, b, a]) / e_ijab

In [ ]:
print(f"Opposite-spin MP2 energy: {e_mp2_os:20.12f}")
print(f"Same-spin MP2 energy:     {e_mp2_ss:20.12f}")
print(f"MP2 energy:               {e_mp2_os + e_mp2_ss:20.12f}")

VeloxChem has its own implementation of the MP2 energy correction. We can check our result against it.

In [ ]:
mp2drv = vlx.Mp2Driver()
mp2drv.compute_conventional(mol, basis, scfdrv.mol_orbs)

np.testing.assert_allclose(e_mp2_os + e_mp2_ss, mp2drv.e_mp2, atol=1e-9)

## Size consistency

We see that to second order, Møller-Plesset perturbation theory only involves up to double excitations from the HF reference. It would thus be natural to consider it an approximation to CISD, and expect it to suffer from the same issue, namely a lack of size consistency. Yet this is not the case. This can easily be verified numerically on a small case

In [ ]:
h2o_2_xyz = """6
2 water 100Å apart                                                                                                            
O    0.000000000000        0.000000000000        0.000000000000                         
H    0.000000000000        0.740848095288        0.582094932012                         
H    0.000000000000       -0.740848095288        0.582094932012
O  100.000000000000        0.000000000000        0.000000000000                         
H  100.000000000000        0.740848095288        0.582094932012                         
H  100.000000000000       -0.740848095288        0.582094932012
"""

mol = vlx.Molecule.from_xyz_string(h2o_2_xyz)

basis = vlx.MolecularBasis.read(mol, "cc-pvdz")

scfdrv = vlx.ScfRestrictedDriver()
scfdrv.compute(mol, basis)

mp2drv = vlx.Mp2Driver()
mp2drv.compute_conventional(mol, basis, scfdrv.mol_orbs)

In [ ]:
print("MP2 Energy correction of 2 water molecules",mp2drv.e_mp2)
print("Twice the energy of 1 water molecule",(e_mp2_os + e_mp2_ss)*2)

We can see that the two energies match. MP2 is size consistent! Why is it behaving better than CISD in this aspect?

The key is that in MP2, the coefficients of the excited determinants are independent of the system size. Thus a molecule would have the same MP2 energy correction regardless of the presence or not of another, non-interacting, molecule. By contrast, in CISD, the coefficients depend on the system size through normalisation, lowering the weight of these determinants as the size of the system increases.

## Further reading

- Shavitt, I.; Bartlett, R. J. *Many-Body Methods in Chemistry and Physics: MBPT and Coupled-Cluster Theory*, Cambridge Molecular Science; Cambridge University Press, 2009.
- Helgaker, T.; Jørgensen, P.; Olsen, J. *Molecular Electronic-Structure Theory* Wiley, 2000.